# **Shelfie: The Library Assistant Bot**
*by ISNA (Izhaan Zubair, Skyler Fernandes, Navya Khurana, Arjun Prabakaran)*

### **Problem Statement**

University and public libraries often house extensive collections of books, but students and patrons struggle to efficiently navigate these collections through traditional search portals. Current systems typically rely on keyword-based search with limited natural language understanding, making it difficult for users to:

* Discover books by author, title, or genre

* Understand availability without manual filtering

* Make reservations or join waitlists seamlessly

* Get personalized or popular recommendations

Additionally, most library systems do not leverage conversational AI to offer a more intuitive, human-like interaction for book exploration and management.

### **Goal**

To build a conversational AI assistant ("Shelfie") that interacts with a library database using natural language, allowing users to search, reserve, and waitlist books through chat — powered by an LLM with SQL and tool-calling capabilities.

### **Key Features**
* Natural language search for books by title, author, or genre

* Real-time reservation of available copies 

* Automated waitlist management for unavailable books 

* Book recommendations based on borrowing history

* Optional Google Search grounding for summaries and reviews

### **GEN AI Capabilities Leveraged**
* **Agents**: The chatbot is an agent that follows a langgraph that defines the flow of conversation and interacts with the library database to reserve/waitlist books for the user.

* **Few-shot prompting**: Used to populate the library database with popular books, their authors, genres and descriptions.

* **Function calling**: Used to understand and query the database to answer questions the user has about the library books and their availability.

* **Structured output/JSON mode/controlled generation**: Used when prompting the LLM to generate the list of books in a structured JSON format that is easily interpretable by our code and added to the library database.

* **Google Search Grounding**: Used to fetch book ratings from Google, Amazon and GoodReads.

##### **Code begins here:**

Run all neccessary pip installs

In [ ]:
%pip uninstall -qqy jupyterlab 
# Remove unused conflicting packages
%pip install -U -q "google-genai==1.7.0"
%pip install faker
%pip install dotenv
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
%pip install -qU "langgraph==0.3.21" "langchain-google-genai==2.1.2" "langgraph-prebuilt==0.1.7"

Do all the neccessary imports to declutter code below. Also verify genai is being imported correctly.

In [ ]:
import sqlite3
import random
from datetime import date
from dataclasses import dataclass
from faker import Faker
from google import genai
from google.genai import types
from google.api_core import retry
from typing import Final
from IPython.display import Image
from pprint import pprint
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages.ai import AIMessage
from typing import Literal
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langchain_core.messages.tool import ToolMessage
import getpass
print(genai.__version__)

Set up `GOOGLE_API_KEY` for both Kaggle and local development.

In [ ]:
import os
from dotenv import load_dotenv

try:
    from kaggle_secrets import UserSecretsClient
    IS_KAGGLE = True
except ImportError:
    IS_KAGGLE = False

# Load environment variables from .env file for local setup
if not IS_KAGGLE:
    load_dotenv()

# Fetch the Google API key
if IS_KAGGLE:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
else:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError("Google API key not found. Please set it in the appropriate environment.")

In [ ]:
NUM_BOOKS: Final[int] = 30
RESERVED_BOOKS: Final[str] = "reserved_books"
WAITLIST: Final[str] = "waitlist"
TOOL_NAME_FIELD: Final[str] = "name"

In [ ]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

In [ ]:
# Remove existing database file (if it exists)
db_file = "library.db"
if os.path.exists(db_file):
    os.remove(db_file)

In [ ]:
@dataclass
class Book:
    name: str
    genre: str
    description: str
    author: str

In [ ]:
def generate_books_list(num_books: int) -> list[Book]:
    """
    Generate a list of books with their details using Google's Gemini API.
    This function ensures the specified number of books is generated by repeatedly
    invoking the API until the desired count is achieved. Each book includes a name,
    genre, description, and author. The description is approximately 1000 characters long.
    The function raises an exception if the requested number of books is less than 1.
    A structured output configuration is used to validate the response format.

    :param num_books: The number of books to generate.
    :type num_books: int
    :return: A list of Book objects.
    :rtype: list[Book]
    :raises Exception: If the requested number of books is less than 1.
    :raises ValueError: If the Google API key is not set.
    :raises genai.errors.*: If there is an error with the API call.
    :raises sqlite3.Error: If there is an error with the SQLite database.
    """
    prompt: str = """
    Generate a list of exactly {num_books} books of any genre. For each book, provide the name, genre, a description of about 1000 characters, and the author.
    Do not include the author's name in the description. Return the result as a JSON array of book objects with the keys: "name", "genre", "description", and "author".
    Examples of a "name", "genre", "description", and "author" are provided below:
    1) "To Kill a Mockingbird", "Fiction", "A gripping tale set in the racially charged American South, exploring themes of justice, morality, and the loss of innocence. Perfect for readers who appreciate profound social commentary interwoven with compelling storytelling.", "Harper Lee"
    2) "1984", "Dystopian", "A chilling vision of a totalitarian future where surveillance and control pervade every aspect of life. Ideal for fans of thought-provoking and cautionary tales.", "George Orwell"
    """

    if num_books < 1:
        raise Exception("Need to generate at least one Book.")
    
    prompt = prompt.format(num_books = num_books)
    client = genai.Client(api_key=GOOGLE_API_KEY)
    
    structured_output_config = types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema={
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "name": {"type": "string"},
                    "genre": {"type": "string"},
                    "description": {"type": "string"},
                    "author": {"type": "string"}
                },
                "required": ["name", "genre", "description", "author"]
            }
        },
        temperature=0.3
    )

    response_len = 0

    while response_len != num_books:
        chat = client.chats.create(model='gemini-2.0-flash')
        response = chat.send_message(
          message=prompt,
          config=structured_output_config,
        )
        book_objects = [Book(**book) for book in response.parsed]
        response_len = len(book_objects)
    
    return book_objects

Do a basic check to see that an exception is raised when the user provides `num_books` < 1.

In [ ]:
try:
    generate_books_list(0)
except:
    print("Expected error when calling generate_books_list with 0 books.\n\n")

Call `generate_books_list` with `NUM_BOOKS`. The time it takes to run the line of code is non-deterministic. This is the number of books produced by gemini is not consitent.

In [ ]:
books_list = generate_books_list(NUM_BOOKS)

Do some validation on the generated output. Verify if the generated list is indeed of length 30. If it's not try running it again. Print the top 5 for inspection.

In [ ]:
assert(len(books_list) == NUM_BOOKS)
assert(all([True for book in books_list if type(book) == Book]))
pprint(books_list[:5])

In [ ]:
# Initialize Faker instance
fake = Faker()

# Create SQLite database and tables
conn = sqlite3.connect("library.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS Users (
    user_id INTEGER PRIMARY KEY,
    name TEXT,
    email TEXT,
    password_hash INT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Book (
    book_id INTEGER PRIMARY KEY,
    name TEXT,
    genre TEXT,
    description TEXT,
    author TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Stock (
    stock_id INTEGER PRIMARY KEY AUTOINCREMENT,
    book_id INTEGER,
    FOREIGN KEY (book_id) REFERENCES Book(book_id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS BorrowingHistory (
    entry_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    stock_id INTEGER,
    borrowed_on DATE,
    returned_on DATE,
    due_by DATE,
    FOREIGN KEY (user_id) REFERENCES Users(user_id),
    FOREIGN KEY (stock_id) REFERENCES Stock(stock_id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Reservations (
    reservation_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    stock_id INTEGER,
    status TEXT CHECK(status in ("Active","Fulfilled","Expired")),
    last_updated DATE,
    reserved_on DATE,
    FOREIGN KEY (user_id) REFERENCES Users(user_id),
    FOREIGN KEY (stock_id) REFERENCES Stock(stock_id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Waitlist (
    book_id INTEGER,
    waitlisted_on DATE,
    FOREIGN KEY (book_id) REFERENCES Book(book_id)
)
""")

passwords = ["ISNA2018", "ISNA2019", "ISNA2020", "ISNA2021", "ISNA2022"]
# Generate and insert sample data for Users table
for user_id in range(1, 6):
    cursor.execute("""
    INSERT INTO Users (user_id, name, email, password_hash)
    VALUES (?, ?, ?, ?)
    """, (user_id, fake.name(), fake.email(), hash(passwords[user_id - 1])))

# Insert real books into Book table
for book_id, book in enumerate(books_list, start=1):
    cursor.execute("""
    INSERT INTO Book (book_id, name, genre, description, author)
    VALUES (?, ?, ?, ?, ?)
    """, (book_id, book.name, book.genre, book.description, book.author))

# Generate and insert sample data for Stock table
for book_id in range(1, 31):
    num_books = random.randint(3, 5)
    j = 0
    while (j < num_books):
        cursor.execute("""
        INSERT INTO Stock (book_id)
        VALUES (?)
        """, (book_id, ))
        j += 1

# Generate and insert sample data for BorrowingHistory table
for entry_id in range(1, 21):
    cursor.execute("""
    INSERT INTO BorrowingHistory (entry_id, user_id, stock_id, borrowed_on, returned_on, due_by)
    VALUES (?, ?, ?, ?, ?, ?)
    """, (entry_id, random.randint(1, 5), random.randint(1, 50), fake.date_between(start_date="-1y", end_date="-1d"), fake.date_between(start_date="+1d", end_date="+1y"), fake.date_between(start_date="+1d", end_date="+1y")))

# Generate and insert sample data for Reservations table
for reservation_id in range(1, 21):
    cursor.execute("""
    INSERT INTO Reservations (reservation_id, user_id, stock_id, status, last_updated, reserved_on)
    VALUES (?, ?, ?, ?, ?, ?)
    """, (reservation_id, random.randint(1, 5), random.randint(1, 50), random.choice(["Active", "Fulfilled", "Expired"]), fake.date_between(start_date="-1y", end_date="today"), fake.date_between(start_date="-1y", end_date="today")))

# Generate and insert sample data for Waitlist table
for book_id in range(1, 6):
    cursor.execute("""
    INSERT INTO Waitlist (book_id, waitlisted_on)
    VALUES (?, ?)
    """, (book_id, fake.date_between(start_date="-1y", end_date="today")))

# Commit changes and close connection
conn.commit()
conn.close()

print("Sample data has been inserted into the SQLite database.")


In [ ]:
conn = sqlite3.connect("library.db")
cursor = conn.cursor()

# Query all tables to display the data
cursor.execute("SELECT * FROM Users")
print("Users:")
for row in cursor.fetchall():
    print(row)

cursor.execute("SELECT * FROM Book")
print("\nBooks:")
for row in cursor.fetchall():
    print(row)

cursor.execute("SELECT * FROM Stock")
print("\nStock:")
for row in cursor.fetchall():
    print(row)

cursor.execute("SELECT * FROM BorrowingHistory")
print("\nBorrowingHistory:")
for row in cursor.fetchall():
    print(row)

cursor.execute("SELECT * FROM Reservations")
print("\nReservations:")
for row in cursor.fetchall():
    print(row)

cursor.execute("SELECT * FROM Waitlist")
print("\nWaitlist:")
for row in cursor.fetchall():
    print(row)

In [ ]:
class BookState(TypedDict):
    """
    State representing the customer's conversation about borrowing library books. State includes:
    - messages: List of messages exchanged in the conversation.
    - user_id: ID of the user interacting with the chatbot.
    - reserved_books: List of book IDs that the user wants to reserve.
    - waitlist: List of book IDs that the user wants to be on the waitlist for.
    - finished: Flag indicating whether the conversation has completed.
    """

    messages: Annotated[list, add_messages]
    user_id: int
    reserved_books: list[tuple[int, str]]
    waitlist: list[tuple[int, str]]
    finished: bool

SYSTEM_INSTRUCTION: Final[str] = (
    "You are Shelfie, a friendly and knowledgeable library assistant chatbot.\n"
    "You help users explore, reserve, and waitlist books available in our library through this chat interface.\n"
    "You have access to a structured SQL database containing detailed information about physical books, borrowing history, current reservations, users, people on the waitlist, etc.\n\n"

    "Your capabilities include:\n"
    "- Searching for books by title, author, genre, or user-provided descriptions and themes. The genre might not match exactly as the user specified. Try searching for something similar if that is the case.\n"
    "- Checking if a physical copy of a book (identified by stock ID) is currently available.\n"
    "   - If available, offer to reserve the book for 24 hours for physical pickup. You must always ask the user to confirm before placing the reservation.\n"
    "   - Once confirmed, proceed to place the reservation on the smallest available stock ID.\n"
    "   - If no stock is available for that book (i.e., all stock IDs are reserved or borrowed), offer to add the user to the waitlist.\n"
    "   - Again, ask for confirmation before placing the user on the waitlist.\n"
    "- Recommending popular or frequently borrowed books, using borrowing history and current availability.\n"
    "- Suggesting alternatives if a requested book is unavailable.\n"
    "- Providing a popularity-based rating out of 5, using data from external reviews (e.g., Google, Amazon, Goodreads). You can search on Google for these reviews.\n\n"
    "- DO NOT USE the internet for anything else apart from searching for book reviews.\n"

    "To retrieve and reason over library data, you can use SQL tools. You interact with an SQL database, and you have access to the following tools:\n"
    "- list_tables: View all available tables in the library database. Use this to check which tables are available for use.\n"
    "- describe_table: Understand the schema of a specific table. Make sure to fully understand how the table is set up.\n"
    "- execute_query: Issue SELECT queries to fetch data needed to respond to user questions.\n\n"
    "PLEASE USE 'list_tables' and 'describe_table' before any user input so you can understand the schema  "
    "Further, use 'execute_query' to provide accurate information. Show your reasoning step by step to the user.\n"

    "When a user expresses interest in reserving a book, your job is to \n"
    "(1) check availability, \n"
    "(2) ask the user to confirm whether they want to proceed, and \n"
    "(3) once they confirm, immediately add the book to their order using `add_to_reserved_list` or `add_to_waitlist`. "
    "Then, automatically call `place_order` to finalize the reservation in the system. ALWAYS confirm the order explicitly with the user before completing it using the 'confirm_order' tool."
    "You must never delay placing the order until the user asks again — the system should use the 'place_order' tool immediately upon receiving confirmation from the 'confirm_order' tool. Clearly inform the user that the reservation or waitlist has been placed." 

    "The reservation logic relies on the stock_id and the waitlist logic relies on book_id:\n"
    "- Users reserve specific physical copies of books by the stock ID.\n"
    "- A user can reserve a book if a stock of the book is available.\n"
    "- If there are multiple books in stock, always pick the smallest available stock ID for reservations. Do not expose the stock ID to the user.\n"
    "- If no stock is available, users can be added to the waitlist using book ID.\n"
    "- Only add a user to the waitlist if no stock is available for the requested book.\n"
    "- Users must never see stock IDs or book IDs. Instead, always refer to books by their title. You can query the title from the database if needed.\n"
    "- Always confirm user intent before modifying reservations or waitlists.\n"
    "- Once the user confirms, immediately perform the reservation or waitlist operation and inform the user it has been successfully placed. Do not wait for them to ask again.\n"
    "- Never assume that a user wants to reserve or waitlist a book unless they clearly confirm. Your flow should involve: (1) check availability, (2) ask for confirmation, (3) perform the reservation or waitlist operation right away, (4) confirm to the user that the action is completed.\n"

    "Your responses should be helpful, concise, and strictly focused on library-related topics.\n"
    "If the user asks something outside the library’s scope, politely decline and redirect them.\n"
    "If a requested feature or tool isn’t implemented yet, say: 'This feature isn't available yet, but we're working on it!'\n"
    "Ask clarifying questions whenever the user’s intent is unclear or ambiguous."
)


WELCOME_MSG: Final[str] = (
    "Welcome to SHELFIE. I can help you find and recommend books with the ability to explore, reserve or waitlist books with ease.\n"
    "You can search by title, description, author, or an excerpt. Let's find your next great read!\n"
    "You must login to reserve or waitlist books. Please provide your user ID and password to login.\n"
    "You can type `q`, `quit`, `exit`, `goodbye` to quit chatting with me anytime but that would make me sad :(\n"
)

In [ ]:
# Create SQLite database and tables
conn = sqlite3.connect("library.db", check_same_thread=False)
cursor = conn.cursor()

def list_tables() -> list[str]:
    """
    Retrieve the names of all tables in the database.

    :return: A list of table names.
    :rtype: list[str]
    :raises sqlite3.Error: If there is an error accessing the database.
    :raises Exception: If the database connection is not established.
    :raises TypeError: If the result is not iterable.
    :raises ValueError: If the result is empty or not a list of strings.
    :raises RuntimeError: If the database connection is closed or not valid.
    """
    
    print(' - DB CALL: list_tables()')

    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]


list_tables()

In [ ]:
def describe_table(table_name: str) -> list[tuple[str, str]]:
    """
    Look up the table schema of a given table in the database.

    :param table_name: The name of the table to describe.
    :return: A list of columns, where each entry is a tuple of (column, type).
    :rtype: list[tuple[str, str]]
    :raises Exception: If the database connection is not established.
    :raises TypeError: If the result is not iterable.
    :raises ValueError: If the result is empty or not a list of tuples.
    :raises RuntimeError: If the database connection is closed or not valid.
    :raises sqlite3.*: If the table does not exist or the query fails.
    """

    print(f' - DB CALL: describe_table({table_name})')

    cursor = conn.cursor()

    cursor.execute(f"PRAGMA table_info({table_name});")

    schema = cursor.fetchall()
    return [(col[1], col[2]) for col in schema]


describe_table("Users")

In [ ]:
def execute_query(sql: str) -> list[list[str]]:
    """
    Execute an SQL statement, returning the results.

    :param sql: The SQL statement to execute.
    :return: The results of the query.
    :rtype: list[list[str]]
    :raises Exception: If the database connection is not established.
    :raises TypeError: If the result is not iterable.
    :raises ValueError: If the result is empty or not a list of lists.
    :raises RuntimeError: If the database connection is closed or not valid.
    :raises sqlite3.*: If there is an error executing the SQL statement.
    """
    print(f' - DB CALL: execute_query({sql})')

    cursor = conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall()


execute_query("SELECT * FROM Users")

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

def maybe_route_to_login_verify(state: BookState) -> Literal["login_verify", "__end__"]:
    if state.get("finished", False):
        return END
    else:
        return "login_verify"

def login_with_welcome_msg(state: BookState) -> BookState:
    """
    The Login node with welcome message. Present the user with a welcome message and ask for their user ID and password.

    :param state: The current state of the conversation.
    :return: The updated state with the welcome message and user ID and password prompts.
    :rtype: BookState
    """
    defaults = {"user_id": None, "messages": [], "reserved_books": [], "waitlist": [], "finished": False}
    
    print(WELCOME_MSG)
    user_input = input("Enter your user ID or {'q', 'quit', 'exit', 'goodbye'} to quit:")

    if user_input.lower() in {"q", "quit", "exit", "goodbye"}:
        return defaults | { "finished": True }

    # Continue the conversation with the Gemini model.
    return defaults | { "user_id": user_input }


def login_verify(state: BookState) -> BookState:
    """
    The node that verifies the credentials against the database.

    :param state: The current state of the conversation.
    :return: The updated state with the success or failure message.
    :rtype: BookState
    """
    user_id = state["user_id"]

    password = hash(getpass.getpass("Password: "))

    # Verify the username and password against the database.
    cursor.execute("SELECT password_hash, name FROM Users WHERE user_id = ?", (user_id,))
    result = cursor.fetchone()
    failure = "Invalid username or password. Please try again or register with us. Registration is currently closed.:("

    if result and result[0] == password:
        success = f"Login successful! Welcome {result[1]} (id: {user_id}). How can I assist you today?"
        print(success)
        return state | {"user_id": user_id}
    else:
        print(failure)
        return state | {"user_id": None}

In [ ]:
read_only_tools = [list_tables, describe_table, execute_query]

@tool
def add_to_reserved_list(stock_id: int, book_name: str) -> str:
    """
    Adds a physical book (stock) to the user's reserved list.

    Args:
        stock_id (int): The ID of the book stock to reserve.
        book_name (str): The name of the book to reserve.

    Returns:
        Returns the updated reserved book list.

    """
    
@tool
def remove_from_reserved_list(stock_id: int, book_name: str):
    """
    Removes a book stock from the user's reserved list.

    Args:
        stock_id (int): The ID of the reserved stock to remove.
        book_name (str): The name of the book to remove from reservations.

    """

@tool
def add_to_waitlist(book_id: int, book_name: str) -> str:
    """
    Adds a book to the user's waitlist.

    Args:
        book_id (int): The ID of the book to waitlist.
        book_name (str): The name of the book to get added to the waitlist for.

    Returns:
        Returns the updated waitlist.
    """

@tool
def remove_from_waitlist(book_id: int, book_name: str) -> str:
    """
    Removes a book from the user's waitlist.

    Args:
        book_id (int): The ID of the book to remove from waitlist.
        book_name (str): The name of the book to remove from the waitlist.

    """

@tool
def confirm_order() -> str:
    """
    Confirms the current reservation and waitlist selections by displaying it to the user and asking for their confirmation. This does NOT place the order.

    Returns:
        Returns the user's free text response
    """

@tool
def place_order() -> str:
    """
    Finalizes the order and places it in the system. Adds all of the user's reserved books and waitlist book to the database.

    Returns:
        Returns an informative message informing the user of their position in the waitlist or reminding them to collect their reserved book within 24 hours.
    """
   
update_tools = [add_to_reserved_list, remove_from_reserved_list, add_to_waitlist, remove_from_waitlist, confirm_order, place_order]

tools_node = ToolNode(read_only_tools)
google_search_tool = types.Tool(google_search=types.GoogleSearch())
llm_with_tools = llm.bind_tools(read_only_tools + update_tools + [google_search_tool.schema])

In [ ]:
def maybe_route_to_chatbot(state: BookState) -> Literal["chatbot", "login"]:
    """
    Route the conversation to the chatbot node if the user is logged in, else go back to login.

    :param state: The current state of the conversation.
    :return: The next node to route to.
    :rtype: Literal["chatbot", "login"]
    """
    if state.get("user_id"):
        # User is logged in, route to the chatbot node.
        return "chatbot"
    else:
        return "login"


def chatbot_node(state: BookState) -> BookState:
    """
    The chatbot with tools. A simple wrapper around the model's own chat interface. It updates the messages with the response from the LLM.

    :param state: The current state of the conversation.
    :return: The updated state with the chatbot's response.
    :rtype: BookState
    """

    if state["messages"]:
        # If there are messages, continue the conversation with the Gemini model.
        new_output = llm_with_tools.invoke([SYSTEM_INSTRUCTION] + state["messages"])
    else:
        # If there are no messages, start with the welcome message.
        new_output = AIMessage(content="How can I help you today?")

    # Set up some defaults if not already set, then pass through the provided state,
    # overriding only the "messages" field.
    return state | {"messages": [new_output]}


def human_node(state: BookState) -> BookState:
    """
    The human node where the user interacts with the chatbot. It sets the user_id to None when
    the user wants to Logout. It also sets finished state to True when the user wants to exit
    the chatbot interface.

    :param state: The current state of the conversation.
    :return: The updated state with the user's input.
    :rtype: BookState
    """
    last_msg = state["messages"][-1]
    print("SHELFIE:", last_msg.content)
    
    user_input = input(f"User({state['user_id']}):")
    if user_input.lower() == "logout":
        state["user_id"] = None
    elif user_input.lower() in {"q", "quit", "exit", "goodbye"}:
        state["finished"] = True
    
    return state | {"messages": [("user", user_input)]}

In [ ]:
def maybe_route_to_human(state: BookState) -> Literal["human", "tools", "update_db"]:
    """
    This function:
    - Routes the conversation to the human node if the chatbot needs user input.
    - Routes to the tools node if a tool is called.
    - Routes to the update_db node if a state needs to be updated.

    :param state: The current state of the conversation.
    :return: The next node to route to.
    :rtype: Literal["human", "tools", "update_db"]
    """
    if not (msgs := state.get("messages", [])):
        raise ValueError(f"No messages found when parsing state: {state}")

    msg = msgs[-1]

    if hasattr(msg, "tool_calls") and len(msg.tool_calls) > 0:
        if any(
            tool["name"] in tools_node.tools_by_name.keys() for tool in msg.tool_calls
        ):
            return "tools"
        else:
            return "update_db"

    else:
        return "human"

In [ ]:
# Define a mapping of tool names to function names
tool_name_mapping_dict: dict[str, str] = {
    "ADD_TO_RESERVED_BOOKS": "add_to_reserved_list",
    "REMOVE_FROM_RESERVED_BOOKS": "remove_from_reserved_list",
    "ADD_TO_WAITLIST": "add_to_waitlist",
    "REMOVE_FROM_WAITLIST": "remove_from_waitlist",
    "CONFIRM_ORDER": "confirm_order",
    "PLACE_ORDER": "place_order",
}


def tool_name_mapping(name: str) -> str:
    """
    Map the tool name to the corresponding function name.
    
    :param name: The name of the tool.
    :return: The mapped function name.
    :rtype: str
    """
    return tool_name_mapping_dict.get(name)


def maybe_exit(state: BookState) -> Literal["login","chatbot", "__end__"]:
    """
    This function:
    - Routes the conversation to the end node if the conversation is finished.
    - Routes to the login node if the user is logged out.
    - Routes to the chatbot node if the user is logged in and the conversation is not finished.

    :param state: The current state of the conversation.
    :return: The next node to route to.
    :rtype: Literal["login","chatbot", "__end__"]
    """

    if state.get("finished", False):
        return END
    elif state.get("user_id") is None:
        return "login"
    else:
        return "chatbot"

def update_db_node(state: BookState) -> BookState:
    """
    This function processes the tool calls made by the user and updates the database accordingly.
    It handles adding and removing books from the reserved list and waitlist, confirming the order, and placing the order.
    It also generates appropriate responses for each action.

    :param state: The current state of the conversation.
    :return: The updated state.
    :rtype: BookState
    """
    
    tool_msg = state.get("messages", [])[-1]
    outbound_msgs: list[AIMessage] = []
    order: dict[str, list[tuple[int,str]]] = {
        RESERVED_BOOKS: state.get(RESERVED_BOOKS, []),
        WAITLIST: state.get(WAITLIST, []),
    }

    boiler_plate_msg: Final[str] = "\n\nReminder: You can make reservations, get added to a waitlist for other books or confirm your order to finalize the changes."

    for tool_call in tool_msg.tool_calls:
        response: str = ""

        if tool_call[TOOL_NAME_FIELD] == tool_name_mapping("ADD_TO_RESERVED_BOOKS"):
            # Add the book to the order
            stock_id = tool_call["args"]["stock_id"]
            book_name = tool_call["args"]["book_name"]
            if (stock_id, book_name) in order.get(RESERVED_BOOKS, []):
                response = f"You have already reserved the book with name {book_name}.{boiler_plate_msg}"
            else:
                order.get(RESERVED_BOOKS, []).append((stock_id, book_name))
                response = f"A reservation for book {book_name} with {stock_id} has been added to your order.{boiler_plate_msg}"
        
        elif tool_call[TOOL_NAME_FIELD] == tool_name_mapping("REMOVE_FROM_RESERVED_BOOKS"):
            # Remove the book from the order
            stock_id = tool_call["args"]["stock_id"]
            book_name = tool_call["args"]["book_name"]
            if (stock_id, book_name) not in order.get(RESERVED_BOOKS, []):
                response = f"You have not reserved the book with name {book_name}. Would you like to make the reservation?{boiler_plate_msg}"
            else:
                order.get(RESERVED_BOOKS, []).remove((stock_id, book_name))
                response = f"A reservation for book {book_name} with {stock_id} has been removed from your order.{boiler_plate_msg}"
        
        elif tool_call[TOOL_NAME_FIELD] == tool_name_mapping("ADD_TO_WAITLIST"):
            # Add the book to the waitlist
            book_id = tool_call["args"]["book_id"]
            book_name = tool_call["args"]["book_name"]
            if (book_id, book_name) in order.get(WAITLIST, []):
                response = f"You are already on the waitlist for the book with name {book_name}.{boiler_plate_msg}"
            else:
                order.get(WAITLIST, []).append((book_id, book_name))
                response = f"You have been added to the waitlist for the book with name {book_name}.{boiler_plate_msg}"

        
        elif tool_call[TOOL_NAME_FIELD] == tool_name_mapping("REMOVE_FROM_WAITLIST"):
            # Remove the book from the waitlist
            book_id = tool_call["args"]["book_id"]
            book_name = tool_call["args"]["book_name"]
            if (book_id, book_name) not in order.get(WAITLIST, []):
                response = f"You are not on the waitlist for the book with name {book_name}. Did you mean to type another book name?{boiler_plate_msg}"
            else:
                order.get(WAITLIST, []).remove((book_id, book_name))
                response = f"You have been removed from the waitlist for the book with name {book_name}.{boiler_plate_msg}"
        
        elif tool_call[TOOL_NAME_FIELD] == tool_name_mapping("CONFIRM_ORDER"):
            # Confirm the order with the user. Display the reserved books and waitlist.
            if not order.get(RESERVED_BOOKS) and not order.get(WAITLIST):
                return_msg: Final[str] = \
                    "You have no books in your order. You can reserve some books or get added to the waitlist \
                        if the book you are looking for is unavailable."
                response = return_msg
            # There are some books either in the reserved books or waitlist. Confirm the order with the user.
            else:
                reserved_books_str = "\n".join([f"{idx}) {book_name} with stock_id: {int(stock_id)}" for idx, (stock_id, book_name) in enumerate(order.get(RESERVED_BOOKS, []), start=1)])
                waitlist_str = "\n".join([f"{idx}) {book_name}" for idx, (_, book_name) in enumerate(order.get(WAITLIST, []), start=1)])
                print("Your order is as follows:\n")
                print(f"Reserved Books:\n{reserved_books_str}\n")
                print(f"Waitlist:\n{waitlist_str}\n")
                response = input("Is this correct? ")
        
        elif tool_call[TOOL_NAME_FIELD] == tool_name_mapping("PLACE_ORDER"):
            # Place the order and update the database.
            reserved_books = order.get(RESERVED_BOOKS, [])
            waitlist = order.get(WAITLIST, [])
            conn = sqlite3.connect("library.db")
            cursor = conn.cursor()
            
            for (stock_id, _) in reserved_books:
                cursor.execute("INSERT INTO Reservations (user_id, stock_id, status, last_updated, reserved_on) VALUES (?, ?, ?, ?, ?)", (state["user_id"], stock_id, "Active", date.today(), date.today()))
            
            for (book_id, _) in waitlist:
                cursor.execute("INSERT INTO Waitlist (book_id, waitlisted_on) VALUES (?, ?)", (book_id, date.today()))

            # Commit the changes to the database.
            conn.commit()
            order.get(RESERVED_BOOKS, []).clear()
            order.get(WAITLIST, []).clear()
            response = "Your order has been placed successfully. Please pick up your reserved books within 24 hours. For the waitlisted books you will be notified when they are available."
        else:
            raise NotImplementedError(f'Unknown tool call: {tool_call[TOOL_NAME_FIELD]}')

        # Record the tool results as tool messages.
        outbound_msgs.append(
            ToolMessage(
                content=response,
                name=tool_call[TOOL_NAME_FIELD],
                tool_call_id=tool_call["id"],
            )
        )
        
    return {"messages": outbound_msgs, "user_id": state["user_id"], "reserved_books": order.get(RESERVED_BOOKS), "waitlist": order.get(WAITLIST), "finished": False}

In [ ]:
# Update the graph to include the modified login node.
graph_builder = StateGraph(BookState)

graph_builder.add_node("login", login_with_welcome_msg)
graph_builder.add_node("login_verify", login_verify)
graph_builder.add_node("chatbot", chatbot_node)
graph_builder.add_node("tools", tools_node)
graph_builder.add_node("human", human_node)
graph_builder.add_node("update_db", update_db_node)

# Add conditional edges from the login node
graph_builder.add_conditional_edges("login_verify", maybe_route_to_chatbot)
graph_builder.add_conditional_edges("chatbot", maybe_route_to_human)
graph_builder.add_conditional_edges("human", maybe_exit)
graph_builder.add_conditional_edges("login", maybe_route_to_login_verify)

# Add edges
graph_builder.add_edge(START, "login")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("update_db", "chatbot")
# Compile the updated graph
chat_graph = graph_builder.compile()

# Visualize the updated graph
Image(chat_graph.get_graph().draw_mermaid_png())

In [ ]:
state = chat_graph.invoke({ "messages": [] }, {"recursion_limit": 500})